In [1]:
import sqlite3 
import pandas as pd
import numpy as np
import pymysql
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import time
import requests
import re
from tqdm import tqdm

In [3]:
import requests
from tqdm import tqdm

day = ["mon", "tue", "wed", "thu", "fri", "sat", "sun"]
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/120.0.0.0 Safari/537.36'}

webtoon_list = []
writers_list = []
painters_list = []
novel_origin_authors_list = []

for day_code in tqdm(day):
    url = f'https://comic.naver.com/api/webtoon/titlelist/weekday?week={day_code}&order=user'
    
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        
        # 웹툰 하나씩 꺼내기 (변수명 i -> webtoon 으로 변경)
        for webtoon in data["titleList"]:
            
            # 공통 ID (Foreign Key 역할)
            t_id = int(webtoon["titleId"])
            
            # 웹으로 바로 이동할 수 있는 url
            site_url = f'https://comic.naver.com/webtoon/list?titleId={t_id}&tab={day_code}'
            
            # 1. 메인 웹툰 정보 (불필요한 작가 문자열 제거)
            webtoon_list.append({
                "titleId": t_id,
                "titleName": webtoon["titleName"],
                "url": site_url,
                "thumbnailUrl": webtoon["thumbnailUrl"],
                "up": webtoon["up"],
                "rest": webtoon["rest"],
                "bm": webtoon["bm"],
                "adult": webtoon["adult"],
                "starScore": float(webtoon["starScore"]),
                "viewCount": int(webtoon["viewCount"]),
                "openToday": webtoon["openToday"],
                "potenUp": webtoon["potenUp"],
                "bestChallengeLevelUp": webtoon["bestChallengeLevelUp"],
                "finish": webtoon["finish"],
                "new": webtoon["new"]
            })
            
            # 2. 글 작가 (titleId 추가 필수!)
            if webtoon.get("writers"):
                for writer in webtoon["writers"]:
                    writers_list.append({
                        "titleId": t_id,      # <--- 이게 있어야 연결됩니다!
                        "writerId": int(writer["id"]),
                        "name": writer["name"],
                        "type": "Writer"      # 구분용 (선택사항)
                    })
            
            # 3. 그림 작가 (titleId 추가 필수!)
            if webtoon.get("painters"):
                for painter in webtoon["painters"]:
                    painters_list.append({
                        "titleId": t_id,      # <--- 이게 있어야 연결됩니다!
                        "painterId": int(painter["id"]),
                        "name": painter["name"],
                        "type": "Painter"     # 구분용 (선택사항)
                    })
                    
            # 4. 원작 작가 (titleId 추가 필수!)
            if webtoon.get("novelOriginAuthors"):
                for origin_author in webtoon["novelOriginAuthors"]:
                    novel_origin_authors_list.append({
                        "titleId": t_id,      # <--- 이게 있어야 연결됩니다!
                        "originAuthorId": int(origin_author["id"]),
                        "name": origin_author["name"],
                        "type": "Original"    # 구분용 (선택사항)
                    })

    else:
        print("에러:", response.status_code)

print(f"웹툰 수집: {len(webtoon_list)}개")
print(f"글 작가 정보: {len(writers_list)}개")

100%|██████████| 7/7 [00:00<00:00, 18.38it/s]

웹툰 수집: 766개
글 작가 정보: 801개


In [ ]:
# engine = create_engine('sqlite:///mydatabase.db')

In [4]:
webtoon_df = pd.DataFrame(webtoon_list).drop_duplicates()
writers_df = pd.DataFrame(writers_list).drop_duplicates()
painters_df = pd.DataFrame(painters_list).drop_duplicates()
novelOriginAuthors_df = pd.DataFrame(novel_origin_authors_list).drop_duplicates()

In [5]:
webtoon_df.to_csv("naver_webtoon.csv", index=False)
writers_df.to_csv("naver_writers.csv", index=False)
painters_df.to_csv("naver_painters.csv", index=False)
novelOriginAuthors_df.to_csv("naver_novelOriginAuthors.csv", index=False)

In [6]:
webtoon_df = pd.read_csv("naver_webtoon.csv")
webtoon_df = webtoon_df.drop(["up", "rest", "bm", "starScore", "viewCount", "openToday", "potenUp", "bestChallengeLevelUp", "new"], axis=1)
webtoon_df

,titleId,titleName,url,thumbnailUrl,adult,finish
0,839004,만남어플 중독,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/839004...,True,False
1,844058,신체,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/844058...,True,False
2,758037,참교육,https://comic.naver.com/webtoon/list?titleId=7...,https://image-comic.pstatic.net/webtoon/758037...,False,False
3,822657,환생천마,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/822657...,False,False
4,844984,20주년 명작 극장,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/844984...,False,False
...,...,...,...,...,...,...
761,800101,헬스던전,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/800101...,False,False
762,836948,우리의 공백,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/836948...,False,False
763,837417,요괴삼월,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/837417...,False,False
764,797155,킬링킬러,https://comic.naver.com/webtoon/list?titleId=7...,https://image-comic.pstatic.net/webtoon/797155...,False,False


In [7]:
writers_df = pd.read_csv("naver_writers.csv")
painters_df = pd.read_csv("naver_painters.csv")
novelOriginAuthors_df = pd.read_csv("naver_novelOriginAuthors.csv")

In [8]:
writers_df

,titleId,writerId,name,type
0,839004,332797,루즌아,Writer
1,844058,355269,엄세윤,Writer
2,758037,301243,채용택,Writer
3,822657,348256,JP,Writer
4,844984,2,조석,Writer
...,...,...,...,...
758,800101,358218,도베도베,Writer
759,836948,387060,조예빈,Writer
760,837417,282041,이윤희,Writer
761,797155,330099,아백,Writer


In [9]:
wirters_pivot = (
    writers_df
    .groupby(["titleId", "type"])["name"]
    .agg(lambda s: ", ".join(sorted(set(s))))
    .unstack(fill_value="")         # role을 컬럼으로 피벗
    .reset_index()
)

webtoon_df = webtoon_df.merge(wirters_pivot, on="titleId", how="left")

In [10]:
painters_pivot = (
    painters_df
    .groupby(["titleId", "type"])["name"]
    .agg(lambda s: ", ".join(sorted(set(s))))
    .unstack(fill_value="")         # role을 컬럼으로 피벗
    .reset_index()
)

webtoon_df = webtoon_df.merge(painters_pivot, on="titleId", how="left")

In [11]:
original_pivot = (
    novelOriginAuthors_df
    .groupby(["titleId", "type"])["name"]
    .agg(lambda s: ", ".join(sorted(set(s))))
    .unstack(fill_value="")         # role을 컬럼으로 피벗
    .reset_index()
)

webtoon_df = webtoon_df.merge(original_pivot, on="titleId", how="left")

In [ ]:
# webtoon_df.to_sql("mytable", con=engine, if_exists="replace")

728

In [12]:
# 1. 데이터를 담을 5개의 그릇 준비 (DB 테이블 구조와 1:1 매칭)
detail_list = []        # 줄거리 등 기본 정보 (1:1)
genre_list = []         # 장르 (1:N)
keyword_tag_list = []   # 단순 텍스트 태그 (1:N) - gfpAdCustomParam 안의 tags
weekday_list = []       # 요일 (1:N)
curation_tag_list = []  # 큐레이션 태그 상세 정보 (1:N)

# 2. 중복된 ID 제거 (API 호출 횟수 줄이기)
unique_ids = webtoon_df['titleId'].unique()

# 3. 세션 사용 (속도 향상 팁: 매번 연결을 새로 맺지 않고 재사용)
session = requests.Session()
session.headers.update(headers)

for page_num in tqdm(unique_ids):
    url = f'https://comic.naver.com/api/article/list/info?titleId={page_num}'
    
    try:
        response = session.get(url) # requests.get 대신 session.get 사용
        
        if response.status_code == 200:
            data = response.json()
            t_id = int(data["titleId"])
            
            # ---------------------------------------------------
            # [1] 메인 상세 정보 (Synopsis)
            # ---------------------------------------------------
            detail_list.append({
                "titleId": t_id,
                "synopsis": data.get("synopsis", ""), # 없을 경우 대비
                # 필요한 다른 정보가 있다면 여기 추가
            })
            
            # gfpAdCustomParam 데이터 가져오기 (없을 수도 있으니 get 사용)
            gfp_data = data.get("gfpAdCustomParam", {})
            
            # ---------------------------------------------------
            # [2] 장르 (Genre) - 리스트 풀어서 저장
            # ---------------------------------------------------
            # 예: ['DRAMA', 'ROMANCE'] -> 각각 저장
            if gfp_data.get("genreTypes"):
                for genre in gfp_data["genreTypes"]:
                    genre_list.append({
                        "titleId": t_id,
                        "genre": genre
                    })
                    
            # ---------------------------------------------------
            # [3] 텍스트 태그 (Tags) - 리스트 풀어서 저장
            # ---------------------------------------------------
            # 예: ['사이다', '먼치킨'] -> 각각 저장
            if gfp_data.get("tags"):
                for tag in gfp_data["tags"]:
                    keyword_tag_list.append({
                        "titleId": t_id,
                        "tag": tag
                    })

            # ---------------------------------------------------
            # [4] 요일 (Weekdays) - 리스트 풀어서 저장
            # ---------------------------------------------------
            if gfp_data.get("weekdays"):
                for day in gfp_data["weekdays"]:
                    weekday_list.append({
                        "titleId": t_id,
                        "day": day
                    })

            # ---------------------------------------------------
            # [5] 큐레이션 태그 (Curation Tags) - titleId 필수 추가!
            # ---------------------------------------------------
            if data.get("curationTagList"):
                for tag_obj in data["curationTagList"]:
                    curation_tag_list.append({
                        "titleId": t_id,         # <--- 핵심: 연결고리 추가
                        "tagId": tag_obj.get("id"),
                        "tagName": tag_obj.get("tagName"),
                        "urlPath": tag_obj.get("urlPath"),
                        "curationType": tag_obj.get("curationType")
                    })
                    
        else:
            print(f"ID {page_num} 에러: {response.status_code}")
            
    except Exception as e:
        print(f"ID {page_num} 접속 중 예외 발생: {e}")

# --- 결과 확인 (판다스 변환) ---
df_detail = pd.DataFrame(detail_list)
df_genre = pd.DataFrame(genre_list)
df_keyword = pd.DataFrame(keyword_tag_list)
df_weekday = pd.DataFrame(weekday_list)
df_curation = pd.DataFrame(curation_tag_list)

df_detail.to_csv("naver_detail.csv", index=False)
df_genre.to_csv("naver_genre.csv", index=False)
df_keyword.to_csv("naver_keyword.csv", index=False)
df_weekday.to_csv("naver_weekday.csv", index=False)
df_curation.to_csv("naver_curation.csv", index=False)

print("수집 완료!")
print(f"상세정보: {len(df_detail)}개")
print(f"장르정보: {len(df_genre)}개")
print(f"태그정보: {len(df_keyword)}개")

100%|██████████| 732/732 [00:33<00:00, 21.74it/s]

수집 완료!
상세정보: 732개
장르정보: 732개
태그정보: 6560개


In [ ]:
df_detail = pd.read_csv("naver_detail.csv")
df_genre = pd.read_csv("naver_genre.csv")
df_keyword = pd.read_csv("naver_keyword.csv")
df_weekday = pd.read_csv("naver_weekday.csv")
df_curation = pd.read_csv("naver_curation.csv")

In [13]:
webtoon_df = webtoon_df.merge(df_detail, on="titleId", how="left")

In [14]:
kw_agg = (
    df_keyword
    .groupby("titleId")["tag"]
    .agg(lambda s: ", ".join(sorted(set(s))))
    .reset_index()
)

# 2) 장르 df와 merge
df_genre_kw = df_genre.merge(kw_agg, on="titleId", how="left")  # genre, tag 두 컬럼

# 3) genre 컬럼에 장르 + 키워드 합치기
def merge_genre_keywords(row):
    g = str(row["genre"]).strip()          # DRAMA, ACTION ...
    t = str(row["tag"]).strip()
    if not t or t == "nan":
        return g
    return f"{g}, {t}"

df_genre_kw["genre"] = df_genre_kw.apply(merge_genre_keywords, axis=1)

# 키워드 컬럼이 더 필요 없으면
df_genre_kw = df_genre_kw.drop(columns=["tag"])
df_genre_kw

,titleId,genre
0,839004,"DRAMA, 고자극드라마, 고자극스릴러, 나쁜남자, 드라마, 미스터리, 범죄, 성인..."
1,844058,"THRILL, 드라마, 범죄, 블랙코미디, 성인웹툰, 스릴러, 이능력, 자극적인, ..."
2,758037,"ACTION, 다크히어로, 먼치킨, 블루스트링, 사이다, 사회고발, 액션, 자극적인..."
3,822657,"HISTORICAL, 고인물, 동양, 동양풍판타지, 먼치킨, 무협/사극, 사이다, ..."
4,844984,"COMIC, 개그, 열혈병맛개그"
...,...,...
727,800101,"ACTION, 4차원, 먼치킨, 사이다, 액션, 이세계"
728,836948,"PURE, 감성적인, 구원서사, 로맨스, 재회, 청춘로맨스, 학원물"
729,837417,"FANTASY, 동양풍로맨스, 러브코미디, 로맨틱코미디, 순진남, 판타지, 현대, ..."
730,797155,"ACTION, 범죄, 복수극, 시리어스, 액션, 이능력, 이능력배틀물, 자극적인, ..."


In [15]:
webtoon_df = webtoon_df.merge(df_genre_kw, on="titleId", how="left")

In [16]:
wk_agg = (
    df_weekday
    .groupby("titleId")["day"]
    .agg(lambda s: ", ".join(sorted(set(s))))
    .reset_index()
)

df_week = webtoon_df.merge(wk_agg, on="titleId", how="left") 
df_week

,titleId,titleName,url,thumbnailUrl,adult,finish,Writer,Painter,Original,synopsis,genre,day
0,839004,만남어플 중독,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/839004...,True,False,루즌아,루즌아,NaN,20살 비인기 스트리머 초롱. 어플로 만난 정체불명의 남자와의 관계 속에서 주변 사...,"DRAMA, 고자극드라마, 고자극스릴러, 나쁜남자, 드라마, 미스터리, 범죄, 성인...","목, 월"
1,844058,신체,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/844058...,True,False,엄세윤,정썸머,NaN,"사채업자, 재벌, 꿈에 그리던 첫사랑까지... 모두가 내 몸을 원한다. 대체 왜?","THRILL, 드라마, 범죄, 블랙코미디, 성인웹툰, 스릴러, 이능력, 자극적인, ...",월
2,758037,참교육,https://comic.naver.com/webtoon/list?titleId=7...,https://image-comic.pstatic.net/webtoon/758037...,False,False,채용택,한가람,NaN,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!\n<부활남>...,"ACTION, 다크히어로, 먼치킨, 블루스트링, 사이다, 사회고발, 액션, 자극적인...",월
3,822657,환생천마,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/822657...,False,False,JP,부겸,장영훈,"철혈의 맹주, 강호의 절대자 '천하진'. 가문의 수치라 불리는 망나니 '벽리단'의 ...","HISTORICAL, 고인물, 동양, 동양풍판타지, 먼치킨, 무협/사극, 사이다, ...",월
4,844984,20주년 명작 극장,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/844984...,False,False,"범배, 조석","범배, 조석",NaN,네이버웹툰이 어느덧 20주년을 맞이했다! 그간 어떤 작품들이 네이버웹툰을 빛냈을까?...,"COMIC, 개그, 열혈병맛개그","금, 월, 일, 화"
...,...,...,...,...,...,...,...,...,...,...,...,...
761,800101,헬스던전,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/800101...,False,False,도베도베,채종,NaN,"인류 최강의 헬스남 한솔은 데드리프트 도중 정신을 잃었는데, 눈을 떠보니 이세계에 ...","ACTION, 4차원, 먼치킨, 사이다, 액션, 이세계",일
762,836948,우리의 공백,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/836948...,False,False,조예빈,조예빈,NaN,누군가 떠나는것이 두려워 회피형으로 자란 '희진'\n자신과 정반대의 모습이 된 소꿉...,"PURE, 감성적인, 구원서사, 로맨스, 재회, 청춘로맨스, 학원물",일
763,837417,요괴삼월,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/837417...,False,False,이윤희,이윤희,NaN,전 태권도 국가대표 선수 이은비. 슬럼프에 빠져 운동을 그만두고 그런대로 살아가던 ...,"FANTASY, 동양풍로맨스, 러브코미디, 로맨틱코미디, 순진남, 판타지, 현대, ...",일
764,797155,킬링킬러,https://comic.naver.com/webtoon/list?titleId=7...,https://image-comic.pstatic.net/webtoon/797155...,False,False,아백,아백,NaN,"10년 전, 살인마에게 부모님을 잃은 주인공 이시우. 마침내 부모님을 죽인 살인마 ...","ACTION, 범죄, 복수극, 시리어스, 액션, 이능력, 이능력배틀물, 자극적인, ...",일


In [17]:
df_week.to_csv("naver_webtoons.csv", index=False)